In [1]:
import sys
import socket
import binascii
from Crypto.Util import strxor
from Crypto.Cipher import AES

In [1]:
"""
performs the xor of string a and b (every character is treated as an 8-bit value)
"""
def xor(a,b):
    return strxor.strxor(a,b)
    
#AES encryption of message <m> with ECB mode under key <key>
def aes_encrypt(message, key):
    obj = AES.new(key, AES.MODE_ECB,'')
    return obj.encrypt(message)
    
#AES decryption of message <m> with ECB mode under key <key>
def aes_decrypt(message, key):
    obj = AES.new(key, AES.MODE_ECB,'')
    return obj.decrypt(message)

def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(message)
    response=''
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(9000)
        if not data: break
        response = response+data
    s.close()
    return response

# Exercise 1
We can use the first 120 bits of IV_CFB as IV_CTR and one of the T_i for CTR will be T_0 of CFB. So we have M_0. Continue in tis way to get all M_i for CFB.

In [2]:
C1='135f75da86be0d8e74d481f793c46dc4b14dc7ec755525cb13e48d8e168c88b25ac2e6cf08d7fa11ab9fd01e2d717a1b1ec619767b31a9f0772ef4ff15506576cd86f1aceea02549610ae860b2d220834e51df8476f607bbdd9fbb3889e1e9a9fc62266e2a226ac106660e3a845987b524d50a1e7e7729788a28952509329f185080432c0492b4b929fb65df12fec6732e5be92cb8452e4b21046558c9a1a2452c98118b91ba8fc38764ea70752899750f6ad61210ef41f85d7ec1a6e390828047f82168755f57674504ed0576fcb0c0eecbb5ffe8cfb2d8f06938cba53f548ba791cf73c8b2634a712ea704a9f10e4d7df8fe3c605fb2e194f32e7540beb9737593f031686b7c251d012507d2da4e37976a3f275763672b84299d5d0c2175e1afd0bda3c125fc01c3614f3cb7831ffbf73f3bdabb571926944ded67ed2f31a26d2c8fa91e86271ebb44d2c0cdfccaa7c7afc5d913a003f3c0dfcc709729fd491d2e29269a0c557ac273bd4fb71def190502c91b7ec970d034591f3e7fd6707191328df18d4bb16dbd0c41ca94c5659232cb3e3fff3123d81d45651e'

In [2]:
def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(message)
    response=''
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(9000)
        if not data: break
        response = response+data
    s.close()
    return response

def chunks(string, length):
    return list(string[0+i:length+i] for i in range(0, len(string), length))

In [6]:
cfb = chunks(C1, 32)
cfb[-1]

'32cb3e3fff3123d81d45651e'

In [29]:
cfb = chunks(C1, 32)
sciper = '253734'
m = '0'*(32*256)
server = 'lasecpc28.epfl.ch'
port = 6666
T = []
M = []

In [9]:
len(cfb[-1])

24

In [31]:
for iv in cfb[:-1]:
    iv_ctr = iv[:30]
    message = sciper + ' ' + iv_ctr + ' ' + m + '\n'
    ctr = chunks( connect_server( server, port, message).rstrip(), 32 )
    i = int(str(iv[30:32]), 16)
    T.append(ctr[i])

In [32]:
for i in range(1, len(cfb)):
    c = cfb[i]
    t = T[i-1]
    if(i < len(cfb)-1):
        temp = int(c, 16) ^^ int(t, 16)
        m = binascii.a2b_hex("{:32x}".format(temp))
    else:
        temp = int(c, 16) ^^ int(t[:24], 16)
        m = binascii.a2b_hex("{:24x}".format(temp))
        
    M.append(m)

In [33]:
''.join(M)

'Three presupposes for the Elven-kings under the sky, Seven for the Dwarf-lords in their halls of stone, Nine for Mortal Men doomed to die, One for the Dark Lord on his dark throne. In the Land of Mordor where the Shadows lie. One presuppose to rule them all, One presuppose to find them, One presuppose to bring them all and in the darkness bind them. In the Land of Mordor where the Shadows lie.'

# Exercise 2
Backtrace and Invert AES to get K. Then implemet forward implementation.

In [147]:
IV21 = "81cfb4b401265b208770983a7413c27b"
P21 = "otherwise you may just spoil eve"
C21 = "ed33655df363f972439210c6eb76bb56ac62459b100f042c115100e3e749f8ca"
IV22 = "a8de1f925258b75833f29720b121b854"
C22 = "e7b63ff33c3c97375d97b74dde53dd74d90e56fe06bd33df3fdc03fb71b4d31f1f73bfb612888e846f5ac150b51421525ebe1b35c68c5cfab6f6b17ad71dfaee8409eb1608e4f35898ac0186d9b1cda73a98f0a4d02dc32f36f79c4d42eb69a64f6e55efab19cdd9a396ee51186367f3bdad478447351c2fe147737aa703b7ceb6309b58f994d09ef7f8"

In [ ]:
binKey1 = '{0:256b}'.format(int(C21, 16) ^^ int(binascii.b2a_hex(P21), 16))
binIV1 = '{0:128b}'.format(int(IV21, 16))

for i in range(2000):
    t = '{0:b}'.format(int(binKey1[1], 2) ^^ int(binKey1[4], 2) ^^ int(binKey1[9], 2) ^^ int(binKey1[255], 2))
    binKey1 = t + binKey1
    if(binKey1[:128] == binIV1):
        break
        
Kp1 = '{:32x}'.format(int(binKey1[128:256], 2))
K = aes_decrypt(Kp1.decode('hex'), IV21.decode('hex'))

Kp2 = aes_encrypt(K, IV22.decode('hex'))
binKey2 = '{0:256b}'.format(int(IV22 + binascii.b2a_hex(Kp2), 16))

In [148]:
binKey1 = '{0:256b}'.format(int(C21, 16) ^^ int(binascii.b2a_hex(P21), 16))
binIV1 = '{0:128b}'.format(int(IV21, 16))

In [149]:
for i in range(2000):
    t = '{0:b}'.format(int(binKey1[1], 2) ^^ int(binKey1[4], 2) ^^ int(binKey1[9], 2) ^^ int(binKey1[255], 2))
    binKey1 = t + binKey1
    if(binKey1[:128] == binIV1):
        break

In [150]:
Kp1 = '{:32x}'.format(int(binKey1[128:256], 2))

In [151]:
K = aes_decrypt(Kp1.decode('hex'), IV21.decode('hex'))

In [213]:
Kp2 = aes_encrypt(K, IV22.decode('hex'))
binKey2 = '{0:256b}'.format(int(IV22 + binascii.b2a_hex(Kp2), 16))

In [214]:
for i in range(len(C22)*4-256):
    t = '{0:b}'.format(int(binKey2[0+i], 2) ^^ int(binKey2[2+i], 2) ^^ int(binKey2[5+i], 2) ^^ int(binKey2[10+i], 2))
    binKey2 = binKey2 + t

In [215]:
t1 = int(binKey2, 2)
t2 = int( C22[:], 16 )
test = t1^^t2

In [216]:
binascii.a2b_hex( '{:x}'.format(test) )

"Oh and one more thing I forgot: when you'll be trying our Enigma machine, try to encrypt the word maneuver and send the result back to me."

# Exercise 3
## Methodology
To figure out how the cipher works, I first queried all 'a' strings of different lengths. This is because according to the encoding this is the all zeros string, and the ciphertext for the all zeros string would just be a function of the key because the plaintext is zero. This might give us some clue about the structure of the key. I made the following observations on the results of the all 'a' queries:
* The letter before ':' always coincides with the first letter after ':'. for this plaintext.
* all the other characters in the ciphertext are always 'a'.

Since the cipher is a blackbox, we have to make some assumptions about it to crack it. I made the following assumptions about the cipher based on the above observations:
* The single character before : in the ciphertext is the IV.
* The IV influences only the first character of the ciphertext.

Then I gave some arbitrary non zero strings as queries, but changed only a part of the string at the time to see how the ciphertext changes with changes in different parts of the plaintext. I noticed that changing the last character of the plaintext only affected the last character of the ciphertext. The first character also changes as the IV is different for each query. More generally, changing the ith character of the plaintext only changed the ith and (i+1)th character of the ciphertext. This means that if we have the first t character of the plaintext, we can get iterate over all possibilities of the (t+1)th character of the plaintext to see the encryption of which one matches with the given cipher. The only problem is the first character, which depends on the IV. Based on these observations, I made the following assumptions about the cipher:
* Changing the (i)th character of the plaintext changes only the (i)th and (i+1)th character of the ciphertext.
* The first character of the ciphertext is only a function of the IV and the first character of the plaintext.

If all the four assumptions we have made about the ciphertext are true, we can get the full plaintext as described in the previous paragraph, subject to figuring out the first character of the ciphertext. Since by our assumption the first character of the ciphertext only depends on the IV and first character of the plaintext, I try to get several tuples of (first character of cipher text, IV, first character of plaintext) to try and find a relation, and indeed I am successful. For the 5-6 examples I try, it seems to hold modulo 29 that:
$$(IV - \text{first character of ciphertext}) = 2*(\text{first character of plaintext})$$

So I try to decrypt based on this assumptions, and fortunately I am successful. If I had not been, I would have had to remove some of the assumptions and try more queries.


In [34]:
C3 = "j:xtgqmvsaqzq.deqtf cjoy,dxwwdhjeodjxzfd.hgcgtsfxysgkhfhkdmghusexuxfsbcoherdbkfqbqoqaeho,qelokrfcjkjmehytf"
P3 = ''

In [2]:
def num(c):
    if(c <= 'z' and c >= 'a'):
        return (ord(c) - ord('a'))
    elif(c == ' '):
        return(26)
    elif(c == '.'):
        return(27)
    elif(c == ','):
        return(28)    

In [15]:
print(num('a'), num('r'), num('r'))
print(num('t'), num(' '), num('r'))
print(num('b'), num('h'), num('f'))

(0, 17, 17)
(19, 26, 17)
(1, 7, 5)


In [60]:
server = 'lasecpc28.epfl.ch'
sciper = '253734'
port = 5555

In [16]:
# For the first character
pnum = (num(' ') - num('r'))%29
pnum

9

In [17]:
(num('t')*2)%29

9

In [87]:
P3 = 'w'
letters = [chr(i) for i in range(ord('a'), ord('z'))]
letters.extend([' ', '.', ','])

In [71]:
print(P3)

w


In [97]:
P3 = 'w'
for i in range(3, len(C3)):
    for j in letters:
        t = P3 + j
        message = sciper + ' ' + t + 'a' + '\n'
        ct = connect_server(server, port, message)
        #print(message, ct[3:-3], C3[3:i+1], i)
        if(ct[3:-3] == C3[3:i+1]):
            print(j)
            P3 = P3 + j
            break
    

h
e
n
 
i
 
f
i
n
d
 
m
y
s
e
l
f
 
i
n
 
t
i
m
e
s
 
o
f
 
t
r
o
u
b
l
e
,
 
c
a
t
a
f
a
l
q
u
e
 
w
i
l
l
 
c
o
m
e
 
t
o
 
m
e
,
 
s
p
e
a
k
i
n
g
 
w
o
r
d
s
 
o
f
 
w
i
s
d
o
m
,
 
c
o
d
e
 
i
n
 
c
.


In [101]:
P3

'when i find myself in times of trouble, catafalque will come to me, speaking words of wisdom, code in c.'

# Exercise 4
Everything is given. Just have to use elliptic curve functionality of sage.

In [116]:
p4 =  6277101735386680763835789423207666416083908700390324961279
a4 =  6277101735386680763835789423207666416083908700390324961276
b4 =  2455155546008943817740293915197451784769108058161191238065
n4 =  6277101735386680763835789423176059013767194773182842284081
Y4 =  (4880704421185356269285731254574038196082295368040177930410, 3252476952228681732222694512787255717022102608249127534923)
d4 =  4530374734388221759604160470592849519021726345429538788071
P4 =  (602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
U4 =  (1355458087547510951659834549953094943930675781843289993571, 5254050291625999318191619702207884939318189720814143813705)
V4 =  (2990739714230682290299609801426676841596844439446139795430, 34535613716520569746498074307772112260047564877561522833)

In [106]:
F = GF(p4)
E = EllipticCurve(F, [a4, b4])

In [121]:
P = E(P4[0], P4[1])
Y = E(Y4[0], Y4[1])
U = E(U4[0], U4[1])
V = E(V4[0], V4[1])

In [126]:
Up = (U*d4).xy()
Up = E(Up[0], Up[1])

In [135]:
Qf = (V-Up).xy()
Qf

(1700383555601412820715483961945651336938783244735256757257,
 5292332824455204956095717751076762428742566090633460301859)

In [136]:
Q = E(Qf[0], Qf[1])
Q

(1700383555601412820715483961945651336938783244735256757257 : 5292332824455204956095717751076762428742566090633460301859 : 1)

In [ ]:
F = GF(29)
#Then we create the Elliptic curve E
E = EllipticCurve(F, [3,1])
print E

#To check whether a point (x,y) is on the curve, call E.is_on_curve(x,y)
print "is the point (1,2) on the curve?",  E.is_on_curve(1,2)
print "is the point (26,20) on the curve?",  E.is_on_curve(26,20)

#To create a point P with coordinates (x,y) on E, simply call E(x,y)
P = E(26,20)
#To print a point P call P.xy()
print "The coordinates of P are", P.xy()

#To add two points P,Q call + operator
Q = E(1,11)
print "Q =", Q.xy()
print "P+Q =", (P+Q).xy()

#To multiply a point P by a constant l, call l*P
print "5Q =", (5*Q).xy()

#To obtain the point at infinity call E(0)
O = E(0)
print "Point at infinity O =", O #Not possible to call for x,y coordinates!
#To check whether a point is the point at infinity, call is_zero() function
print "Is point Q the point at infinity? ", Q.is_zero()
print "Is point O the point at infinity? ", O.is_zero()

#Compute the order of the curve. WARNING CAN BE SLOW
print "The order of E is",E.order()

#Given a x coordinate, it's possible to list all points on the curve that have this x coordinate with the function lift_x and the parameter all=True
print "The possible points (in projective form) when x = 26 are",  E.lift_x(26, all=True)
print "The possible points (in xy() form) when x = 26 are",  map(lambda u: u.xy(),E.lift_x(26, all=True))

# Exercise 5
Just draw diagram

In [9]:
C5 = "113ac90cf74391150225e9bd696a5378d1304e91b06d007cb0b5eabe7b96d40a"

In [10]:
cl = C5[:32].decode('hex')
cr = C5[32:].decode('hex')

In [16]:
len(C5[32:])

32

In [23]:
import string

In [22]:
for i in range(256):
    h = '{:02x}'.format(i)
    h = h*16
    k = h.decode('hex')
    pl = xor(cl, aes_encrypt(cr, k))
    pr = xor(cr, aes_encrypt(pl, k))
    if(all(ord(c) < 127 and c in string.printable for c in (pl+pr))):
        print(pl + pr)
    

I'd like pomaded with my caviar.
